In [1]:
# -*- coding = utf-8 -*-
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dense

In [2]:
resnet = ResNet50(include_top=False, pooling='avg')
model = Sequential()
model.add(resnet)
model.add(Dense(1))
model.layers[0].trainable = False
model.add(Dense(2, activation='softmax'))
print(model.summary())

24182784/94765736 [======>.......................] - ETA: 14s

KeyboardInterrupt: 

In [3]:
num_skipped = 0
for folder_name in ("beauty", "ugly"):
    folder_path = os.path.join("./train", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [2]:
batch_size = 32
img_height = 180
img_width = 180
epochs = 30
train_dir = './train'
val_dir = './val'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  label_mode='categorical',  
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  label_mode='categorical', 
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 499 files belonging to 2 classes.
Using 400 files for training.
Found 160 files belonging to 2 classes.
Using 32 files for validation.


In [79]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=["accuracy"],
)
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/30
13/13 [==============================] - 13s 992ms/step - loss: 0.2632 - accuracy: 0.5450 - val_loss: 0.2813 - val_accuracy: 0.5000
Epoch 2/30
13/13 [==============================] - 12s 943ms/step - loss: 0.2291 - accuracy: 0.6275 - val_loss: 0.2107 - val_accuracy: 0.5938
Epoch 3/30
13/13 [==============================] - 12s 941ms/step - loss: 0.1979 - accuracy: 0.6975 - val_loss: 0.2130 - val_accuracy: 0.5000
Epoch 4/30
13/13 [==============================] - 12s 939ms/step - loss: 0.1714 - accuracy: 0.7675 - val_loss: 0.2309 - val_accuracy: 0.6562
Epoch 5/30
13/13 [==============================] - 12s 930ms/step - loss: 0.1552 - accuracy: 0.8050 - val_loss: 0.2241 - val_accuracy: 0.6250
Epoch 6/30
13/13 [==============================] - 12s 934ms/step - loss: 0.1439 - accuracy: 0.8375 - val_loss: 0.2337 - val_accuracy: 0.6250
Epoch 7/30
13/13 [==============================] - 12s 935ms/step - loss: 0.1357 - accuracy: 0.8300 - val_loss: 0.2474 - val_accuracy: 0.6250

In [80]:
model.save('./my_model') 

INFO:tensorflow:Assets written to: ./my_model\assets


In [3]:
new_model = tf.keras.models.load_model('./my_model')

In [4]:
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2049      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 4         
Total params: 23,589,765
Trainable params: 23,536,645
Non-trainable params: 53,120
_________________________________________________________________


In [5]:
loss,acc = new_model.evaluate(val_ds, verbose=2)

1/1 - 0s - loss: 0.3218 - accuracy: 0.5312


In [6]:
for image, label in val_ds.take(1):
    print(image.shape)
    print(new_model.predict(image))
    print(label)

(32, 180, 180, 3)
[[8.7555004e-03 9.9124449e-01]
 [2.4796680e-01 7.5203317e-01]
 [9.8856854e-01 1.1431513e-02]
 [5.8338004e-01 4.1661993e-01]
 [6.1147779e-01 3.8852215e-01]
 [1.0362004e-02 9.8963797e-01]
 [8.1447840e-01 1.8552156e-01]
 [9.9449182e-01 5.5081658e-03]
 [4.8433423e-01 5.1566577e-01]
 [1.2863686e-02 9.8713624e-01]
 [9.7942221e-01 2.0577824e-02]
 [3.0286178e-02 9.6971381e-01]
 [4.6362706e-02 9.5363724e-01]
 [9.3641919e-01 6.3580818e-02]
 [2.1895826e-02 9.7810417e-01]
 [1.2623535e-02 9.8737645e-01]
 [9.9168372e-01 8.3162459e-03]
 [4.6207923e-01 5.3792077e-01]
 [2.6292630e-02 9.7370738e-01]
 [9.5587708e-02 9.0441221e-01]
 [8.4683031e-01 1.5316963e-01]
 [9.4011474e-01 5.9885278e-02]
 [9.9531388e-01 4.6861041e-03]
 [4.2423004e-01 5.7576996e-01]
 [9.8548222e-01 1.4517769e-02]
 [5.8915678e-02 9.4108427e-01]
 [8.8444829e-01 1.1555172e-01]
 [5.5573258e-04 9.9944431e-01]
 [5.4307908e-01 4.5692095e-01]
 [8.9512736e-01 1.0487269e-01]
 [8.7180412e-01 1.2819591e-01]
 [8.3849841e-01 1.615

In [7]:
img1 = cv2.imread('./465.jpg')
img2 = cv2.imread('./592.jpg')

In [8]:
img = cv2.resize(img, (180, 180))
re_img = img.reshape([1] + list(img.shape))
print(re_img.shape)
new_model.predict(re_img)

NameError: name 'img' is not defined

In [ ]:
data = []
img1 = cv2.resize(img1, (180, 180))
img2 = cv2.resize(img2, (180, 180))

In [ ]:
data.append(img1)

In [ ]:
data.append(img2)

In [ ]:
n = numpy.array(data)

In [ ]:
n.shape

In [ ]:
x = new_model.predict(n)

In [ ]:
for i, j in x:
    print(i)
    print(j)

In [51]:
from pathlib import Path
import numpy as np
myfile = open('./imgfilelistname.txt', 'r')
datas = myfile.readlines()
imgs = []
for data in datas:
    data = data.rstrip()
    img = cv2.imread(data)
    img = cv2.resize(img, (img_height, img_width))
    imgs.append(img)
imgs = np.array(imgs)

In [36]:
a = './data.jpg'

In [38]:
path = Path(a)

In [39]:
path

WindowsPath('data.jpg')